In [1]:
!pip -q install transformers==4.17
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 590.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installa

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import time

**Dataset Utilization**

In [3]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=7)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [11]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("Best trial:")
# print(study.best_trial)

In [12]:
training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=7,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16 * 2,
    learning_rate=1.1603919360800287e-05,
    weight_decay=0.006845868059871536,
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"<<==============Training Time==================>>")
print(f"Training time: {training_time:.2f} seconds")
print(f"<<=============================================>>")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 224


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.396263,0.836813,0.368771,0.013289,0.025654
1,No log,0.344067,0.868132,0.302780,0.259136,0.279263


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.396263,0.836813,0.368771,0.013289,0.025654
1,No log,0.344067,0.868132,0.302780,0.259136,0.279263
2,No log,0.310855,0.884615,0.637518,0.322259,0.395450
3,No log,0.284469,0.895055,0.822744,0.405316,0.499716
4,No log,0.266329,0.904396,0.794911,0.485050,0.582196
5,No log,0.257876,0.909341,0.795896,0.521595,0.620135
6,No log,0.254137,0.909890,0.795894,0.524917,0.621546


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

<<==============Training Time==================>>
Training time: 314.86 seconds
<<=============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


{'eval_loss': 0.25413748621940613, 'eval_accuracy': 0.9098901098901099, 'eval_precision': 0.7958938811941536, 'eval_recall': 0.5249169435215947, 'eval_f1': 0.6215460965983212, 'eval_runtime': 3.8758, 'eval_samples_per_second': 67.083, 'eval_steps_per_second': 2.322, 'epoch': 6.98}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Model Saving

In [14]:
!mkdir 'best_model_scibert_scivocab_uncased_pharo'
!mkdir 'best_model_scibert_scivocab_uncased_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_scibert_scivocab_uncased_pharo')
tokenizer.save_pretrained('./best_model_scibert_scivocab_uncased_pharo_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_scibert_scivocab_uncased_pharo')
tokenizer = AutoTokenizer.from_pretrained('./best_model_scibert_scivocab_uncased_pharo_tokenizer')

Saving model checkpoint to ./best_model_scibert_scivocab_uncased_pharo
Configuration saved in ./best_model_scibert_scivocab_uncased_pharo/config.json
Model weights saved in ./best_model_scibert_scivocab_uncased_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_scibert_scivocab_uncased_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_scibert_scivocab_uncased_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_scibert_scivocab_uncased_pharo/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_scibert_scivocab_uncased_pharo",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initia

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 289
  Batch size = 32


Evaluation Metrics: {'accuracy': 0.9223924864063272, 'precision': 0.7653045589246453, 'recall': 0.5714285714285714, 'f1': 0.6399507808511901}
Inference Time: 4.29 seconds


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!zip -r best_model_scibert_scivocab_uncased_pharo.zip './best_model_scibert_scivocab_uncased_pharo'
!zip -r best_model_scibert_scivocab_uncased_pharo_tokenizer.zip './best_model_scibert_scivocab_uncased_pharo_tokenizer'

In [19]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_scibert_scivocab_uncased_pharo.zip "/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo/"
!cp best_model_scibert_scivocab_uncased_pharo_tokenizer.zip "/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo_Tokenizer/"

ValueError: mount failed

### Load and Test Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo/best_model_scibert_scivocab_uncased_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/scibert_scivocab_uncased_pharo_Tokenizer/best_model_scibert_scivocab_uncased_pharo_tokenizer.zip' -d './'

In [ ]:
best_model_scibert_scivocab_uncased_pharo = 'best_model_scibert_scivocab_uncased_pharo'
best_model_scibert_scivocab_uncased_pharo_tokenizer = 'best_model_scibert_scivocab_uncased_pharo_tokenizer'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(best_model_scibert_scivocab_uncased_pharo)
tokenizer = AutoTokenizer.from_pretrained(best_model_scibert_scivocab_uncased_pharo_tokenizer)

In [ ]:
text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.2f} seconds")
output

In [ ]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")